# Phase 1: Introduction

## 1.1 Background
Open-source software (OSS) is the backbone of modern technology, driving innovation through community collaboration. A healthy open-source project relies on a continuous influx of new contributors to fix bugs, add features, and maintain the codebase. However, diving into an unfamiliar, complex repository is one of the most daunting tasks for any developer, whether they are junior programmers or seasoned veterans.

## 1.2 Problem Statement
Despite the desire to contribute to open-source, developers face a massive barrier to entry: **onboarding friction**. New contributors are often confronted with massive codebases, fragmented or outdated documentation, and complex architectural dependencies.

Currently, a developer looking to contribute must spend hours or even days manually reading code, tracing functions, and deciphering the project's structure before writing a single line of code. This steep learning curve leads to developer frustration, abandoned pull requests, and a significant loss of potential talent for the open-source community.

## 1.3 Solution & Approach
To eliminate this onboarding friction, I'm building **CodeBuddy**, an AI-powered Open-Source Onboarder. CodeBuddy acts as an intelligent companion that guides developers through new codebases, explaining architecture, locating specific logic, and answering questions in real-time.

***For assignment perspective I'm using local tech stack cannot ingest large repository yet. Once I'm able to mature it as agent running in a dedicated GPU can ingest large reposiroty***

### Our Approach:
I will tackle this problem by building a Retrieval-Augmented Generation (RAG) pipeline tailored for source code:
1. **Data Ingestion & Parsing:** I will ingest the target open-source repository (code files, READMEs, and existing documentation) and parse the codebase into meaningful, logical chunks (e.g., functions, classes, and modules).
2. **Embedding & Indexing:** These code chunks will be converted into vector embeddings and stored in a vector database, allowing for fast, semantic search capabilities.
3. **Retrieval & Generation:** When a user asks a question (e.g., *"Where is the database connection handled?"* or *"Explain how the authentication middleware works"*), the system will retrieve the most relevant code snippets and pass them to an LLM to generate a clear, context-aware explanation.

### Why Use Large Language Models (LLMs)?
Using an LLM is the cornerstone of CodeBuddy because traditional search tools (like `grep` or IDE text search) are vastly insufficient for true code comprehension. I need LLMs for the following reasons:
* **Semantic Code Understanding:** LLMs go beyond keyword matching; they understand the *intent* and logic behind the code, allowing them to explain complex algorithms in plain English.
* **Contextual Synthesis:** An LLM can look at multiple retrieved files simultaneously and synthesize an answer that explains how different components of the repository interact with each other.
* **Interactive Guidance:** LLMs allow developers to have a dynamic, conversational back-and-forth. If a developer doesn't understand an initial explanation, they can ask the LLM to simplify it, provide an example, or trace a specific variable's journey through the codebase.

# Phase 2: System Design

In this phase, we design the architecture of CodeBuddy. We will break this down step-by-step, starting with how the user interacts with the system and how the AI interprets their needs.

## 2.1 Conversation Initialization & Intent Handling

Before CodeBuddy can search the repository or explain code, it needs to establish a structured conversation with the user and accurately understand their goals. This interaction layer acts as the front door to our system.

### Step 1: Start a Conversation (LLM Initialization)
To initiate the conversation, we must instantiate the LLM with a specific "System Prompt" and memory management. This ensures the model behaves like an expert developer rather than a generic chatbot.

* **System Prompting:** We define the LLM's persona, constraints, and instructions. For example: *"You are CodeBuddy, an expert senior engineer. Your goal is to help a junior developer onboard onto a new open-source repository. Be concise, reference specific file paths when known, and do not make up code that does not exist."*
* **Session Memory:** Because onboarding requires continuous dialogue, we must initialize a chat memory object (e.g., using LangChain's `ConversationBufferMemory`). This allows the LLM to remember previous questions and maintain context throughout the session.

### Step 2: Get User Input (Intent Clarity & Confirmation)
When a developer types a question, open-source repositories are often too large to simply search blindly. We need an intent routing mechanism to ensure we are retrieving the right kind of information.

* **Capturing Input:** The system receives the user's raw query (e.g., *"How do I start this?"* or *"Where is the auth logic?"*).
* **Intent Clarity (Classification):** Before querying our vector database, we pass the raw input through a lightweight LLM call (or a dedicated classifier) to determine the *intent* of the question. Common intents for CodeBuddy might include:
    * `PROJECT_SETUP`: Questions about installation, Docker, or `npm install`.
    * `ARCHITECTURE_OVERVIEW`: High-level questions about how services interact.
    * `CODE_EXPLANATION`: Deep dives into specific functions or files.
    * `CONTRIBUTION_GUIDELINES`: Questions about how to submit a PR or run tests.
* **Intent Confirmation:** If a user's query is highly ambiguous (e.g., *"Explain the user stuff"*), the system pauses. Instead of running an expensive and likely inaccurate search, CodeBuddy replies with an **Intent Confirmation** prompt: *"It sounds like you want to know about User Authentication and Authorization. Is that correct, or are you looking for the User UI components?"* This human-in-the-loop step ensures high-accuracy retrieval later in the pipeline.

## 2.1.1 Implementation: Initialization and Intent Routing (Local Model)

In the code block below, we set up the conversational interface entirely on our local machine. This ensures that repository code and developer queries remain completely private.



This implementation handles:
1. **Local LLM Setup:** We initialize a local model (e.g., `llama3.1` or `mistral`) using Ollama. We set the `format="json"` parameter to guarantee the model outputs strictly parsable JSON for our routing logic.
2. **Memory:** We instantiate a `ConversationBufferMemory` to keep track of the chat history.
3. **Intent Classifier:** We define a specific prompt that forces the LLM to analyze the user's input and categorize it, outputting the result, a confidence score, and any clarifying questions.

In [1]:
from langsmith.env import get_git_info
!pip install langchain langchain-ollama

  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.7/604.7 kB 28.8 MB/s eta 0:00:00
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)


In [6]:
import json
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain.memory import ConversationBufferMemory

In [31]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0.2,
    format="json"
)
memory = ConversationBufferMemory(return_messages=True)

intent_template = """
You are the routing engine for CodeBuddy, an AI assistant helping developers onboard onto an open-source codebase.
Analyze the user's input and classify their intent into EXACTLY ONE of the following categories:
- PROJECT_SETUP: Questions about installation, dependencies, Docker, etc.
- ARCHITECTURE_OVERVIEW: Questions about folder structure, data flow, and architecture.
- CODE_EXPLANATION: Questions about specific files, functions, or localized logic.
- AMBIGUOUS: The request is too vague to search the codebase effectively.

User Input: {user_input}

Respond strictly in valid JSON format with the following schema:
{{
  "intent": "CATEGORY",
  "confidence": 0.95,
  "clarifying_question": "Ask a question here ONLY if the intent is AMBIGUOUS, otherwise leave blank"
}}
"""

intent_prompt = PromptTemplate(
    input_variables=["user_input"],
    template=intent_template
)

def process_user_input(user_query):
    print(f"User: {user_query}")

    formatted_prompt = intent_prompt.format(user_input=user_query)

    response = llm.invoke(formatted_prompt)

    try:
        raw_text = response.content if hasattr(response, "content") else response
        classification = json.loads(raw_text)
        intent = classification.get("intent")
        confidence = float(classification.get("confidence", 0))

        print(f"[System Log] Intent Detected: {intent} (Confidence: {confidence})")

        if intent == "AMBIGUOUS" or confidence < 0.7:
            clarification = classification.get("clarifying_question", "Could you please clarify what part of the project you are asking about?")
            print(f"CodeBuddy: {clarification}")
            return None

        return intent

    except json.JSONDecodeError:
        print("[System Log] Error parsing intent. Asking user for clarification.")
        return "AMBIGUOUS"

print("--- Test 1: Clear Intent ---")
intent_1 = process_user_input("How do I spin up the local Postgres database using Docker?")

print("\n--- Test 2: Ambiguous Intent ---")
intent_2 = process_user_input("Explain the user stuff to me.")

--- Test 1: Clear Intent ---
User: How do I spin up the local Postgres database using Docker?
[System Log] Intent Detected: PROJECT_SETUP (Confidence: 0.95)

--- Test 2: Ambiguous Intent ---
User: Explain the user stuff to me.
[System Log] Intent Detected: AMBIGUOUS (Confidence: 0.8)
CodeBuddy: Can you please specify what 'user stuff' refers to? Are you asking about a particular file or functionality?


## 2.2 Dynamic GitHub Ingestion and Vector Database

For CodeBuddy to answer questions about a repository, it first needs to "read" and index the code. Instead of relying on local files, our system will prompt the user for a live GitHub URL and pull the code dynamically.


This phase consists of three main steps:
1. **Dynamic Loading:** We use Python's `input()` to grab the repository URL. Then, LangChain's `GitLoader` automatically clones it and loads its text files into memory.
2. **Chunking (Code-Aware):** We split the files into smaller chunks using a Python-specific text splitter to ensure we don't accidentally slice functions or classes in half.
3. **Embedding & Storage (ChromaDB):** We convert these text chunks into high-dimensional vector representations (embeddings) using our local Ollama model and store them in **Chroma** for semantic searching.

In [20]:
!pip install GitPython chromadb langchain-community langchain-ollama

  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
Using cached smmap-5.0.2-py3-none-any.whl (24 kB)


In [21]:
!pip install -U langchain-ollama

In [33]:
import os
import shutil
from langchain_community.document_loaders import GitLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
import chromadb
import chromadb.config

In [37]:
print("="*50)
print("📥 CodeBuddy Repository Ingestion")
print("="*50)

def get_git_info():
    github_url = input("Enter the GitHub Repository URL to ingest (e.g., https://github.com/hwchase17/langchain-tiny): ").strip()

    if not github_url:
        github_url = "https://github.com/bhagirathbhard/langchain-reporead"
        print(f"\n[System Log] No URL provided. Defaulting to: {github_url}")

    repo_destination = "./cloned_repo"
    db_dir = "./chroma_db"

    if os.path.exists(repo_destination):
        print(f"\n[System Log] Clearing previous repository data at {repo_destination}...")
        shutil.rmtree(repo_destination)


    print(f"\n[System Log] Cloning and loading documents from {github_url}...")

    loader = GitLoader(
        clone_url=github_url,
        repo_path=repo_destination,
        branch="main",
        file_filter=lambda file_path: file_path.endswith(".py") or file_path.endswith(".md")
    )

    documents = loader.load()
    print(f"[System Log] Successfully loaded {len(documents)} file(s) from GitHub.")

    print("\n[System Log] Splitting code into semantic chunks...")
    python_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.PYTHON,
        chunk_size=500,
        chunk_overlap=50
    )
    chunks = python_splitter.split_documents(documents)
    print(f"[System Log] Created {len(chunks)} chunk(s).")

    print("\n[System Log] Generating embeddings and storing in local ChromaDB...")

    embeddings = OllamaEmbeddings(model="llama3.1")

    vector_db = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=db_dir
    )

    print("\n[System Log] ✅ GitHub Ingestion complete! CodeBuddy's knowledge base is ready.")

get_git_info()

📥 CodeBuddy Repository Ingestion

[System Log] Cloning and loading documents from https://github.com/bhagirathbhard/langchain-reporead...
[System Log] Successfully loaded 1 file(s) from GitHub.

[System Log] Splitting code into semantic chunks...
[System Log] Created 4 chunk(s).

[System Log] Generating embeddings and storing in local ChromaDB...

[System Log] ✅ GitHub Ingestion complete! CodeBuddy's knowledge base is ready.


## 2.3 Retrieval and Answer Generation

The final step in our RAG pipeline is taking the user's question, finding the relevant code we just pulled from GitHub, and letting the LLM explain it.

[Image of a Retrieval-Augmented Generation system querying a vector database]

This phase works by chaining together a few core components:
1. **The Retriever:** We convert our ChromaDB into a "Retriever" object. When given a query, it will search the database and return the top `k` most semantically similar code chunks from the GitHub repository.
2. **The Prompt Template:** We define CodeBuddy's persona and provide a strict instruction to *only* use the retrieved context to answer the question, preventing the model from hallucinating code that doesn't exist.
3. **The Document Chain (`create_stuff_documents_chain`):** This LangChain utility takes the retrieved code chunks and "stuffs" them into the `{context}` variable of our prompt.
4. **The Retrieval Chain (`create_retrieval_chain`):** This acts as the orchestrator. It takes the user's input, hits the retriever, passes the documents to the document chain, and returns the final LLM-generated answer.

In [26]:

from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [27]:
llm = OllamaLLM(model="llama3.1", temperature=0.3)

retriever = vector_db.as_retriever(search_kwargs={"k": 4})

system_prompt = (
    "You are CodeBuddy, an expert senior engineer helping a junior developer onboard onto a new open-source codebase. "
    "Use the following pieces of retrieved code context to answer the user's question. "
    "If you don't know the answer or the context doesn't contain the right code, just clearly say that you don't know. "
    "Do not make up code or file names. Keep your explanations clear, concise, and structured.\n\n"
    "Context:\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("--- Testing CodeBuddy Answer Generation ---")

test_question = "Based on the repository, what is the main purpose of this project?"
print(f"User: {test_question}\n")

response = rag_chain.invoke({"input": test_question})

print("CodeBuddy:")
print(response["answer"])

print("\n---")
print(f"[System Log] Answer generated using {len(response['context'])} retrieved source chunks.")

--- Testing CodeBuddy Answer Generation ---
User: Based on the repository, what is the main purpose of this project?

CodeBuddy:
The main purpose of this project is to provide a set of tools and scripts that leverage Langchain and OpenAI to read and analyze the content of a repository. It allows users to ask questions about a repository by storing them in a `questions.txt` file, and then generates individual answers in markdown files within an `answers` folder using the output from the LLM (Large Language Model).

---
[System Log] Answer generated using 4 retrieved source chunks.


## 2.4 Agentic Extension: Function Calling (Tool Use)

RAG pipelines are excellent for answering questions about static code logic, but open-source repositories are dynamic. If a developer asks, *"What are the latest open issues? Has someone raised a PR for this?"*, a static vector database cannot help them.

To solve this, we are upgrading CodeBuddy into an **Agent**. By implementing **Tool Calling**, we give our local LLM the autonomy to execute real Python functions—like making live API calls to GitHub—whenever it detects that the user needs live data.

### The Approach:
1. **The Tool (`@tool`):** We define a Python function that requests the latest issues from the GitHub API. We programmatically distinguish between standard issues and Pull Requests (GitHub returns both in the same endpoint).
2. **The Chat Model (`ChatOllama`):** We use a model class that supports the `.bind_tools()` method.
3. **The Agent Executor:** We use LangChain's `create_tool_calling_agent` to create a loop where the LLM can "think," decide to use the GitHub tool, wait for the Python script to fetch the JSON data, and finally summarize the live results for the user.

In [44]:
!pip install requests

In [45]:
import requests
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

print("="*50)
print("🔌 Upgrading CodeBuddy with GitHub API Tools")
print("="*50)

@tool
def check_github_issues(repo_url: str) -> str:
    """
    Fetches the latest open issues and checks if any Pull Requests have been raised for a given GitHub repository URL.
    Args:
        repo_url: The full GitHub repository URL (e.g., "https://github.com/hwchase17/langchain-tiny")
    """
    try:
        parts = repo_url.rstrip('/').split('/')
        owner, repo = parts[-2], parts[-1]

        api_url = f"https://api.github.com/repos/{owner}/{repo}/issues?state=open&per_page=5"

        headers = {"Accept": "application/vnd.github.v3+json"}
        response = requests.get(api_url, headers=headers)

        if response.status_code != 200:
            return f"GitHub API error: Status {response.status_code}. (You might be rate limited)"

        issues_data = response.json()

        if not issues_data:
            return "There are no open issues or PRs for this repository right now."

        summary = ["Here are the latest 5 open items from the repository:\n"]
        for item in issues_data:
            number = item.get("number")
            title = item.get("title")

            is_pr = "pull_request" in item
            item_type = "Pull Request" if is_pr else "Issue"

            summary.append(f"- [{item_type} #{number}] {title}")

        return "\n".join(summary)

    except Exception as e:
        return f"Tool error fetching issues: {str(e)}"

chat_model = ChatOllama(model="llama3.1", temperature=0.1)

tools = [check_github_issues]

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are CodeBuddy, an expert developer assistant. Use the provided tools to answer the user's questions. Summarize the tool's raw output nicely for the user."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(chat_model, tools, agent_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("\n[System Log] Asking CodeBuddy to check live issues...\n")

test_query = f"Can you check the latest issues for {GITHUB_URL} and tell me if there are any Pull Requests?"
print(f"👤 You: {test_query}\n")

response = agent_executor.invoke({"input": test_query})

print("\n🤖 CodeBuddy:")
print(response["output"])

🔌 Upgrading CodeBuddy with GitHub API Tools

[System Log] Asking CodeBuddy to check live issues...

👤 You: Can you check the latest issues for https://github.com/bhagirathbhard/langchain-reporead and tell me if there are any Pull Requests?



> Entering new AgentExecutor chain...

Invoking: `check_github_issues` with `{'repo_url': 'https://github.com/bhagirathbhard/langchain-reporead'}`


There are no open issues or PRs for this repository right now.<|python_tag|>{"name": "check_github_issues", "parameters": {"repo_url":"https://github.com/bhagirathbhard/langchain-reporead"}}

> Finished chain.

🤖 CodeBuddy:
<|python_tag|>{"name": "check_github_issues", "parameters": {"repo_url":"https://github.com/bhagirathbhard/langchain-reporead"}}


# Phase 3: Bringing It All Together (Unified RAG + Agent UI)

In this final phase, we combine all our system components into a single interactive chat interface.

CodeBuddy now has a "split-brain" architecture:
1. **The Router:** When a user asks a question, the LLM first classifies the intent.
2. **The RAG Path (`rag_chain`):** If the user asks about code logic or architecture, the system queries the local Chroma vector database.
3. **The Agent Path (`agent_executor`):** If the user asks about live PRs or open issues, the system bypasses the database and triggers the GitHub API tool to fetch real-time data.
4. **The UI:** Everything is wrapped in a clean, inline `ipywidgets` interface to prevent IDE popups and maintain chat history.

In [38]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.5 MB/s eta 0:00:00


In [46]:
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain_core.prompts import PromptTemplate

print("="*60)
print("🚀 Compiling Final CodeBuddy Interface...")
print("="*60)

updated_intent_template = """
You are the routing engine for CodeBuddy. Analyze the user's input and classify their intent into EXACTLY ONE of the following categories:
- PROJECT_SETUP: Questions about installation or dependencies.
- ARCHITECTURE_OVERVIEW: Questions about folder structure and data flow.
- CODE_EXPLANATION: Questions about specific files or code logic.
- LIVE_ISSUES: Questions about open issues, pull requests, or repository status.
- AMBIGUOUS: The request is too vague.

User Input: {user_input}

Respond strictly in valid JSON format:
{{"intent": "CATEGORY", "confidence": 0.95, "clarifying_question": "Only if AMBIGUOUS"}}
"""
updated_intent_prompt = PromptTemplate(input_variables=["user_input"], template=updated_intent_template)

def get_intent(user_query):
    formatted_prompt = updated_intent_prompt.format(user_input=user_query)
    response = llm.invoke(formatted_prompt)
    raw_text = response.content if hasattr(response, "content") else response
    try:
        return json.loads(raw_text).get("intent", "AMBIGUOUS")
    except:
        return "AMBIGUOUS"


output_area = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '10px'})
text_input = widgets.Text(
    placeholder='Ask about the code or live issues/PRs...',
    layout=widgets.Layout(width='80%')
)
submit_button = widgets.Button(
    description='Send',
    button_style='primary',
    layout=widgets.Layout(width='15%')
)
input_box = widgets.HBox([text_input, submit_button])

with output_area:
    print("🤖 CodeBuddy: Your Unified Open-Source Onboarder is Ready!")
    print(f"🔗 Connected to: {GITHUB_URL}")
    print("Type 'exit' or 'quit' to end the session.\n" + "-"*60)

def handle_interaction(button_click=None):
    user_query = text_input.value
    if not user_query.strip(): return
    text_input.value = ''

    if user_query.lower() in ['exit', 'quit']:
        with output_area:
            print("\n🤖 CodeBuddy: Chat closed. Happy coding!")
        input_box.close()
        return

    with output_area:
        print(f"\n👤 You: {user_query}")


        intent = get_intent(user_query)
        print(f"   [System Log] Intent routed to: {intent}")

        if intent == "AMBIGUOUS":
            print("🤖 CodeBuddy: Could you please clarify what part of the project you are asking about?")

        elif intent == "LIVE_ISSUES":
            print("   [System Log] Triggering GitHub Agent...\n")
            try:
                response = agent_executor.invoke({"input": user_query})
                print("🤖 CodeBuddy:")
                print(response["output"])
            except Exception as e:
                print(f"🤖 CodeBuddy: Error fetching live data: {e}")

        else:
            print("   [System Log] Searching local vector database...\n")
            try:
                response = rag_chain.invoke({"input": user_query})
                print("🤖 CodeBuddy:")
                print(response["answer"])

                sources = set([doc.metadata.get('source', 'Unknown') for doc in response['context']])
                print("\n📂 Sources referenced:")
                for source in sources: print(f"  - {source}")
            except Exception as e:
                print(f"🤖 CodeBuddy: Error searching codebase: {e}")

submit_button.on_click(handle_interaction)
text_input.on_submit(handle_interaction)
display(output_area, input_box)

🚀 Compiling Final CodeBuddy Interface...


/var/folders/gv/vp9gzv0530z5gsl8qdmlmvm00000gn/T/ipykernel_36945/2853798692.py:103: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text_input.on_submit(handle_interaction)


Output(layout=Layout(border_bottom='1px solid #ccc', border_left='1px solid #ccc', border_right='1px solid #cc…